In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
# import cv2
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
# from tensorflow.keras.layers.normalization import BatchNormalization
import os
import seaborn as sns
# from keras.applications import DenseNet121
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

In [2]:
train_data_path = 'Training'
test_data_path = 'Testing'

In [3]:
x_train = []
y_train = []
x_test = []
y_test = []

In [4]:
# Function to process images in a folder
def process_images(folder_path, images_list, labels_list):
    for class_folder in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_folder)
        if not os.path.isdir(class_path):
            continue
        
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            image = Image.open(image_path)
            
            # Convert the image to grayscale
            image = image.convert('L')
            
            # Resize the image to 64x64
            image = image.resize((64, 64))
            
            # Convert the image to a NumPy array
            image_array = np.array(image)
            
            # Normalize the image data (if needed)
            image_array = image_array / 255.0
            
            # Append the processed image and its label to the lists
            images_list.append(image_array)
            labels_list.append(class_folder)


In [5]:
# Process training images
process_images(train_data_path, x_train, y_train)

# Process testing images
process_images(test_data_path, x_test, y_test)

In [6]:
x_tr=[]
x_ts=[]
y_tr=[]
y_ts=[]

In [7]:
x_tr = np.array(x_train)
y_tr = np.array(y_train)

In [8]:
x_ts = np.array(x_test)
y_ts = np.array(y_test)

In [ ]:
# # Convert the lists to NumPy arrays
# x_train = np.array(x_train)
# x_test = np.array(x_test)

In [9]:
x_tr.shape,y_tr.shape,x_ts.shape,y_ts.shape

((84096, 64, 64), (84096,), (21024, 64, 64), (21024,))

In [10]:
# Convert class labels to categorical format using one-hot encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_tr)
y_test_encoded = label_encoder.transform(y_ts)

onehot_encoder = OneHotEncoder(sparse=False)
y_tr1 = onehot_encoder.fit_transform(y_train_encoded.reshape(-1, 1))
y_ts1 = onehot_encoder.transform(y_test_encoded.reshape(-1, 1))

In [11]:
x_tr.shape,y_tr1.shape,x_ts.shape,y_ts1.shape

((84096, 64, 64), (84096, 5), (21024, 64, 64), (21024, 5))

In [12]:
# Reshape the data by adding an extra dimension
x_tr = np.expand_dims(x_tr, axis=-1)
x_ts = np.expand_dims(x_ts, axis=-1)

In [13]:
x_tr.shape,y_tr1.shape,x_ts.shape,y_ts1.shape

((84096, 64, 64, 1), (84096, 5), (21024, 64, 64, 1), (21024, 5))

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [15]:
# Define the shape of the input data
input_shape = (64, 64, 1)

# Define the number of classes
num_classes = 5

# Define the 2D CNN model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(6, 6), activation='relu', padding='same', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
# model.add(Conv2D(32, kernel_size=(4, 4), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(Flatten())
# Fully connected
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# Output layer
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        1184      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 21, 21, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 21, 21, 32)        0         
                                                                 
 flatten (Flatten)           (None, 14112)             0         
                                                                 
 dense (Dense)               (None, 128)               1806464   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        

In [16]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Fit the model to the training data
history=model.fit(x_tr, y_tr1, batch_size=64, epochs=100)

Epoch 1/100
1314/1314 [==============================] - 100s 76ms/step - loss: 0.9338 - accuracy: 0.6468
Epoch 2/100
1314/1314 [==============================] - 103s 78ms/step - loss: 0.6208 - accuracy: 0.7477
Epoch 3/100
1314/1314 [==============================] - 105s 80ms/step - loss: 0.5326 - accuracy: 0.7778
Epoch 4/100
1314/1314 [==============================] - 101s 77ms/step - loss: 0.4887 - accuracy: 0.7905
Epoch 5/100
1314/1314 [==============================] - 109s 83ms/step - loss: 0.4596 - accuracy: 0.8016
Epoch 6/100
1314/1314 [==============================] - 100s 76ms/step - loss: 0.4384 - accuracy: 0.8084
Epoch 7/100
1314/1314 [==============================] - 101s 77ms/step - loss: 0.4199 - accuracy: 0.8181
Epoch 8/100
1314/1314 [==============================] - 101s 77ms/step - loss: 0.4067 - accuracy: 0.8234
Epoch 9/100
1314/1314 [==============================] - 100s 76ms/step - loss: 0.3903 - accuracy: 0.8305
Epoch 10/100
1314/1314 [======================

In [ ]:
# _, accuracy = model.evaluate(x_ts, y_ts1, batch_size=64, verbose=0)

In [ ]:
# accuracy

In [18]:
prediction = model.predict(x_ts)

657/657 [==============================] - 10s 14ms/step


In [19]:
prediction_NN=np.argmax(prediction, axis=-1)

In [20]:
prediction_NN

array([0, 0, 0, ..., 4, 4, 4], dtype=int64)

In [21]:
y_ts1_1=np.argmax(y_ts1, axis=-1)

In [22]:
y_ts1_1

array([0, 0, 0, ..., 4, 4, 4], dtype=int64)

In [23]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_ts1_1, prediction_NN))

Accuracy =  0.9454908675799086


In [26]:
model.save('M1_all_imgs_1lyr.h5')

In [24]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_ts1_1, prediction_NN)
print(cm)

[[11603    86    12   185    40]
 [  144  1249     1    32   151]
 [   35     0  1498     0     3]
 [  119    39     0   981   122]
 [   47    37     2    91  4547]]


In [25]:
from sklearn.metrics import classification_report

# print(class_labels)
target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4']
print(classification_report(y_ts1_1, prediction_NN, target_names=target_names,digits=3))

              precision    recall  f1-score   support

     Class 0      0.971     0.973     0.972     11926
     Class 1      0.885     0.792     0.836      1577
     Class 2      0.990     0.975     0.983      1536
     Class 3      0.761     0.778     0.769      1261
     Class 4      0.935     0.963     0.949      4724

    accuracy                          0.945     21024
   macro avg      0.908     0.896     0.902     21024
weighted avg      0.945     0.945     0.945     21024



In [ ]:
# model.save("model_2DCNN_New1.1.h5")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
# class_dict = {0: 'Class 0',
#               1: 'Class 1',
#               2: 'Class 2',
#               3: 'Class 3',
#               4: 'Class 4'}

# multiclass = np.array([[11441,    16,    27,   232,   169],
#                        [567,   669,     0,   179,   141],
#                        [780,     1,   774,     0,     1],
#                        [53,     1,     0,  1124,    91],
#                        [5,     0,     0,    48,  4706]])

# fig, ax = plot_confusion_matrix(
#     conf_mat=multiclass,
#     class_names=class_dict.values(),show_normed=True,colorbar=True)